In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
1,COVID-19 Daily Update 11-14-2020 West Virgini...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sat, 14 Nov 2020 15:01:50 GMT",2020-11-14 15:01:50+00:00
4,The Dakotas are 'as bad as it gets anywhere in...,https://www.usatoday.com/story/news/health/202...,"Sat, 14 Nov 2020 14:23:01 GMT",2020-11-14 14:23:01+00:00
3,Governors issue stringent new measures as US r...,https://www.cnn.com/2020/11/14/health/us-coron...,"Sat, 14 Nov 2020 11:35:00 GMT",2020-11-14 11:35:00+00:00
6,Latest on COVID-19 in MN: The deadliest week ...,https://www.mprnews.org/story/2020/11/14/lates...,"Sat, 14 Nov 2020 11:00:00 GMT",2020-11-14 11:00:00+00:00
9,How a Covid-19 vaccine could change travel for...,https://www.cnn.com/travel/article/covid-vacci...,"Sat, 14 Nov 2020 09:50:27 GMT",2020-11-14 09:50:27+00:00
8,Rio samba school has a new mission: fighting C...,https://www.modernhealthcare.com/safety/rio-sa...,"Sat, 14 Nov 2020 06:00:00 GMT",2020-11-14 06:00:00+00:00
7,"Governor Carney, DPH, DEMA Announce Community ...",https://news.delaware.gov/2020/11/13/governor-...,"Sat, 14 Nov 2020 01:09:30 GMT",2020-11-14 01:09:30+00:00
0,"Coronavirus (COVID-19) Update: November 13, 20...",https://www.fda.gov/news-events/press-announce...,"Fri, 13 Nov 2020 22:34:00 GMT",2020-11-13 22:34:00+00:00
2,Governor Cuomo Announces New Record Number of ...,https://www.governor.ny.gov/news/governor-cuom...,"Fri, 13 Nov 2020 22:22:05 GMT",2020-11-13 22:22:05+00:00
5,"Coronavirus (COVID-19) Update: November 12, 20...",https://www.fda.gov/news-events/press-announce...,"Thu, 12 Nov 2020 23:12:00 GMT",2020-11-12 23:12:00+00:00
